In [1]:
import os
from io import BytesIO

import json
import pandas as pd
import numpy as np

import boto3
import sagemaker
from sagemaker.sklearn import SKLearn

from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer
from sagemaker.serverless import ServerlessInferenceConfig

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_client = boto3.client('sagemaker')

In [3]:
s3 = boto3.client('s3')
bucket = 'movielens-recommendation'

In [4]:
train_channel = sagemaker.session.TrainingInput(
    f"s3://{bucket}/data/linucb/All.csv", 
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix"
)
eval_channel = sagemaker.session.TrainingInput(
    f"s3://{bucket}/data/linucb/user_genre_matrix_df.csv", 
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix"
)

In [5]:
script_path = "inference.py"
source_dir = "scripts"
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
estimator = SKLearn(
    framework_version='0.23-1',
    py_version='py3',
    entry_point=script_path,
    source_dir=source_dir,
    instance_type="ml.m5.2xlarge",
    instance_count=1,
    role=role,
    sagemaker_session=sagemaker_session,
    output_path=f"s3://{bucket}/output",
)

In [7]:
estimator.fit({ "train": train_channel, "eval": eval_channel })

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-01-08-10-21-22-975


Using provided s3_resource
2024-01-08 10:21:23 Starting - Starting the training job...
2024-01-08 10:21:46 Starting - Preparing the instances for training.........
2024-01-08 10:23:05 Downloading - Downloading input data...
2024-01-08 10:23:41 Training - Training image download completed. Training in progress...2024-01-08 10:24:00,438 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-01-08 10:24:00,441 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-01-08 10:24:00,478 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-01-08 10:24:00,632 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-01-08 10:24:00,644 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-01-08 10:24:00,655 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-01-08 10:24:00,664 sagemaker-training-toolk

finish training
### SAVING MODEL ###
Shape of A: (19, 119, 119), Data type: float32
Shape of b: (19, 119), Data type: float32
Estimated memory usage of A: 1076236 bytes (1.035003662109375 MB)
Estimated memory usage of b: 9044 bytes (1.035003662109375 MB)
Total estimated memory usage: 1.035003662109375 MB
finish saving
### SAVING CODE ###
saved code in  /opt/ml/model/code/
### SAVING PARAMETER ###
saved parameter at  <_io.TextIOWrapper name='/opt/ml/model/networkParams' mode='w' encoding='UTF-8'>
### TRAINING END ###
2024-01-08 10:26:10,890 sagemaker-containers INFO     Reporting training SUCCESS

2024-01-08 10:26:16 Uploading - Uploading generated training model
2024-01-08 10:26:27 Completed - Training job completed
Training seconds: 202
Billable seconds: 202


In [8]:
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb = 2048,
    max_concurrency = 3
)

In [9]:
endpoint_name = "LinUCB"

In [10]:
try:
    sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    print(f"Deleting existing endpoint: {endpoint_name}")
    sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
except sagemaker_client.exceptions.ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == 'ValidationException':
        print(f"No existing endpoint found with name: {endpoint_name}")
    else:
        raise

Deleting existing endpoint: LinUCB


In [11]:
try:
    sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)
    print(f"Deleted endpoint configuration: {endpoint_name}")
except sagemaker_client.exceptions.ClientError as e:
    print(f"Error deleting endpoint configuration: {e}")

Deleted endpoint configuration: LinUCB


In [12]:
deployed_model = estimator.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    serializer=CSVSerializer(),
    deserializer=JSONDeserializer(),
    serverless_inference_config=serverless_config
)

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-01-08-10-27-08-692
INFO:sagemaker:Creating endpoint-config with name LinUCB
INFO:sagemaker:Creating endpoint with name LinUCB


-----!